In [18]:
import sqlite3
import pandas as pd

df = pd.read_csv("data/clean_SuperStoreOrders.csv")

conn = sqlite3.connect("sales.db")
df.to_sql("sales", conn, if_exists="replace", index=False)

conn.close()



In [ ]:
conn = sqlite3.connect("sales.db")


Finding the total sales per month.

In [7]:
q1 = """
SELECT 
    strftime('%Y-%m', order_date) AS month,
    SUM(sales) AS total_sales
FROM sales
GROUP BY month
ORDER BY month;
"""
pd.read_sql(q1, conn)


,month,total_sales
0,None,4760117.0
1,2011-01,50437.0
2,2011-02,54400.0
3,2011-03,51789.0
4,2011-04,39497.0
5,2011-05,37097.0
6,2011-06,41584.0
7,2011-07,53349.0
8,2011-08,50490.0
9,2011-09,45143.0


Finding the top 10 customers by revenue.

In [8]:
q2 = """
SELECT 
    customer_name, 
    SUM(sales) AS total_sales 
FROM sales
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10;
"""
pd.read_sql(q2,conn)

,customer_name,total_sales
0,Eric Murdock,19489.0
1,John Grady,19483.0
2,Maria Etezadi,18237.0
3,Theone Pippenger,17416.0
4,Dan Reichenbach,16705.0
5,Ben Ferrer,16661.0
6,Randy Bradley,16399.0
7,Mathew Reese,16125.0
8,Muhammed Yedwab,16091.0
9,Steven Ward,15984.0


Finding the average order value.

In [9]:
q3 = """
SELECT
    AVG(sales) AS avg_order_value
FROM sales;
"""
pd.read_sql(q3,conn)

,avg_order_value
0,161.017838


Recognising the repeat customers.

In [11]:
q4 = """
SELECT 
    customer_name,
    COUNT(DISTINCT order_id) AS num_orders
FROM sales
GROUP BY customer_name
HAVING num_orders > 1
ORDER BY num_orders DESC;
"""
pd.read_sql(q4,conn)

,customer_name,num_orders
0,Michael Paige,46
1,Anna Andreadi,46
2,Tracy Blumstein,45
3,Sara Luxemburg,45
4,Patrick O'Brill,45
...,...,...
790,Peter Bühler,19
791,Andrew Allen,19
792,Nicole Brennan,16
793,Michael Oakman,15


Category that gave the most profit.

In [12]:
q5 = """
SELECT 
    category,
    SUM(profit) AS total_profit
FROM sales
GROUP BY category
ORDER BY total_profit DESC;
"""
pd.read_sql(q5, conn)

,category,total_profit
0,Office Supplies,350107.32450
1,Technology,267573.47238
2,Furniture,97049.37790


Finding the most profitable region.

In [13]:
q6 = """
SELECT 
    region,
    SUM(profit)/SUM(sales) AS profit_margin
FROM sales
GROUP BY region
ORDER BY profit_margin DESC;
"""
pd.read_sql(q6, conn)


,region,profit_margin
0,Canada,0.245112
1,North Asia,0.191050
2,Central Asia,0.170603
3,West,0.135261
4,North,0.134871
5,East,0.116358
6,Central,0.093414
7,Oceania,0.087224
8,Caribbean,0.085027
9,South,0.061832


Finding the products that gave losses.

In [14]:
q7 = """
SELECT 
    product_name,
    SUM(profit) AS total_profit
FROM sales
GROUP BY product_name
HAVING total_profit < 0
ORDER BY total_profit;
"""
pd.read_sql(q7, conn)


,product_name,total_profit
0,"Rogers Lockers, Blue",-3.171486e+03
1,"Ikea Library with Doors, Pine",-2.994912e+03
2,"Bevis Computer Table, Fully Assembled",-2.911670e+03
3,"Nokia Smart Phone, Full Size",-2.803955e+03
4,"Hon Conference Table, Adjustable Height",-2.635687e+03
...,...,...
757,"Acco PRESSTEX Data Binder with Storage Hooks, ...",-1.614000e-01
758,"Stiletto Ruler, Serrated",-1.374000e-01
759,Global Super Steno Chair,-4.440892e-15
760,"Avery Trapezoid Extra Heavy Duty 4"" Binders",-3.552714e-15


Calculating the growth rate.

In [15]:
q8 = """
WITH monthly AS (
    SELECT 
        strftime('%Y-%m', order_date) AS month,
        SUM(sales) AS total_sales
    FROM sales
    GROUP BY month
)
SELECT 
    m1.month,
    (m1.total_sales - m2.total_sales) * 1.0 / m2.total_sales AS growth_rate
FROM monthly m1
JOIN monthly m2 
  ON date(m1.month || '-01') = date(m2.month || '-01', '+1 month');
"""
pd.read_sql(q8, conn)


,month,growth_rate
0,2011-02,0.078573
1,2011-03,-0.047996
2,2011-04,-0.237348
3,2011-05,-0.060764
4,2011-06,0.120953
5,2011-07,0.282921
6,2011-08,-0.053591
7,2011-09,-0.105902
8,2011-10,-0.011896
9,2011-11,-0.061785


Finding the most profitable shipping mode.

In [16]:
q9 = """
SELECT 
    ship_mode,
    SUM(profit) AS total_profit
FROM sales
GROUP BY ship_mode
ORDER BY total_profit DESC;
"""
pd.read_sql(q9, conn)


,ship_mode,total_profit
0,Standard Class,435958.12746
1,Second Class,142746.94960
2,First Class,90680.82338
3,Same Day,45344.27434


Calculating the lifetime value of customers.

In [17]:
q10 = """
SELECT 
    customer_name,
    SUM(profit) AS lifetime_profit
FROM sales
GROUP BY customer_name
ORDER BY lifetime_profit DESC
LIMIT 10;
"""
pd.read_sql(q10, conn)


,customer_name,lifetime_profit
0,Katharine Harms,3400.16880
1,Mathew Reese,3105.12260
2,Brian Moss,3028.13338
3,Sanjit Engle,3007.02352
4,John Grady,2870.44010
5,Bill Shonely,2839.77056
6,Larry Tron,2804.89294
7,Jim Karlsson,2791.22160
8,Neil Knudson,2790.40890
9,Laura Armstrong,2773.70620
